In [5]:
"""
Golden‑Bit × Reuleaux Prime Tests
---------------------------------
Run cell‑by‑cell in Colab or execute directly:

    !python reuleaux_prime_tests.py

Step 1 – define Reuleaux constants
Step 2 – classify primes (6k±1)
Step 3 – build weighted Euler product  ζ_r(s)
Step 4 – probe ζ_r and classical ζ on the critical line

Extend later with explicit‑formula or rail‑phase checks.
"""

import mpmath as mp

# -----------------------------------------
# 1.  Geometry constants (50‑d.p.)
mp.mp.dps = 50
A_R = (mp.pi - mp.sqrt(3)) / 2          # Reuleaux area, width 1
r     = A_R / (mp.pi / 2)                # dimensionless weight  ≈0.44879
print("Reuleaux area  A_R =", A_R)
print("Dimensionless weight r =", r)

# -----------------------------------------
# 2.  Prime classification
def classify(p: int) -> int:
    """Return +1 for p ≡ 1 (mod 6), −1 for p ≡ 5 (mod 6)."""
    return 1 if p % 6 == 1 else -1

def primes_upto(N: int):
    """Simple sieve; good enough to 10^7."""
    sieve = bytearray(b"\x01") * (N + 1)
    sieve[0:2] = b"\x00\x00"
    for p in range(2, int(N ** 0.5) + 1):
        if sieve[p]:
            sieve[p * p : N + 1 : p] = b"\x00" * len(range(p * p, N + 1, p))
    return [p for p, isprime in enumerate(sieve) if isprime]

# -----------------------------------------
# 3.  Weighted Euler product
def zeta_weighted(s, cutoff=1_000_000):
    """Evaluate ζ_r(s) by truncated Euler product."""
    prod = mp.mpf(1)
    for p in primes_upto(cutoff):
        eps = classify(p)
        prod *= 1 / (1 - (r ** eps) * p ** (-s))
    return prod

# -----------------------------------------
# 4.  Critical‑line scan
def critical_scan(t_vals, cutoff=1_000_000):
    for t in t_vals:
        s = mp.mpf("0.5") + 1j * mp.mpf(t)
        zr = zeta_weighted(s, cutoff=cutoff)
        z  = mp.zeta(s)
        print(f"t = {t:6.2f} | ζ_r = {str(zr)} | ζ = {str(z)}")

if __name__ == "__main__":
    # quick test near first three non‑trivial zeros
    critical_scan([14.1, 21.0, 25.0], cutoff=200_000)

Reuleaux area  A_R = 0.70477092301045797246759852088681525862718207278236
Dimensionless weight r = 0.44867110457820795048867350168703055860261351963336
t =  14.10 | ζ_r = (-0.081594980343461582581999656583286851780962650415914 + 0.020048009833425405146514559457286713898191313470235j) | ζ = (0.0046984001834891872175818780236912815127512378626733 - 0.027058282374251048230206453673007771520226243205673j)
t =  21.00 | ζ_r = (0.0015012148493468119359870337893256592971989783624424 - 0.00048644824984621743967774805977020530179820670094166j) | ζ = (-0.0051620646381019009048328210143896516987087371570711 - 0.024546964575121902877924957175591007826355344497814j)
t =  25.00 | ζ_r = (0.00067045019872456983295238545939941754728244872709427 - 0.0035857172496023089127697419823274657961627715676605j) | ζ = (0.004984593364035675383362507223590309110587828269388 - 0.014012301962583382962901214587651289145189890620602j)
